<a href="https://colab.research.google.com/github/BobMcDear/Blue-Book-for-Bulldozer/blob/main/blueBookForBulldozers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fastai==0.7
!pip install scikit-learn==0.21
!pip install kaggle

In [ ]:
from fastai.structured import *
from fastai.imports import *
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

In [ ]:
from google.colab import files

In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
# For uploading a Kaggle API token
files.upload()

In [ ]:
# For downloading the dataset
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle
!kaggle competitions download -c bluebook-for-bulldozers

In [ ]:
!unzip Train.zip

### Data Pre-processing

In [ ]:
df_raw = pd.read_csv('Train.csv', low_memory=False, parse_dates=['saledate'])
df_raw.sort_values('saledate', inplace=True)

In [ ]:
df_raw.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,fiModelDesc,fiBaseModel,fiSecondaryDesc,fiModelSeries,fiModelDescriptor,ProductSize,fiProductClassDesc,state,ProductGroup,ProductGroupDesc,Drive_System,Enclosure,Forks,Pad_Type,Ride_Control,Stick,Transmission,Turbocharged,Blade_Extension,Blade_Width,Enclosure_Type,Engine_Horsepower,Hydraulics,Pushblock,Ripper,Scarifier,Tip_Control,Tire_Size,Coupler,Coupler_System,Grouser_Tracks,Hydraulics_Flow,Track_Type,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
205615,1646770,9500,1126363,8434,132,18.0,1974,NaN,NaN,1989-01-17,TD20,TD20,NaN,NaN,NaN,Medium,"Track Type Tractor, Dozer - 105.0 to 130.0 Hor...",Texas,TTT,Track Type Tractors,NaN,OROPS,NaN,NaN,NaN,NaN,Direct Drive,NaN,NaN,NaN,NaN,NaN,2 Valve,NaN,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None or Unspecified,Straight,None or Unspecified,NaN,NaN
92803,1404019,24000,1169900,7110,132,99.0,1986,NaN,NaN,1989-01-31,416,416,NaN,NaN,NaN,NaN,Backhoe Loader - 14.0 to 15.0 Ft Standard Digg...,Florida,BL,Backhoe Loaders,Two Wheel Drive,EROPS,None or Unspecified,None or Unspecified,No,Standard,Standard,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98346,1415646,35000,1262088,3357,132,99.0,1975,NaN,NaN,1989-01-31,12G,12,G,NaN,NaN,NaN,Motorgrader - 130.0 to 145.0 Horsepower,Florida,MG,Motor Graders,No,OROPS,NaN,NaN,NaN,NaN,None or Unspecified,NaN,None or Unspecified,12',None or Unspecified,No,Base + 1 Function,None or Unspecified,None or Unspecified,Yes,Sideshift & Tip,"14""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
169297,1596358,19000,1433229,8247,132,99.0,1978,NaN,NaN,1989-01-31,644,644,NaN,NaN,NaN,Medium,Wheel Loader - 175.0 to 200.0 Horsepower,Florida,WL,Wheel Loader,NaN,OROPS,None or Unspecified,NaN,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 Valve,NaN,NaN,NaN,NaN,None or Unspecified,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
274835,1821514,14000,1194089,10150,132,99.0,1980,NaN,NaN,1989-01-31,A66,A66,NaN,NaN,NaN,NaN,Wheel Loader - 120.0 to 135.0 Horsepower,Florida,WL,Wheel Loader,NaN,OROPS,None or Unspecified,NaN,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 Valve,NaN,NaN,NaN,NaN,None or Unspecified,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional


In [ ]:
df_raw.tail()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,fiModelDesc,fiBaseModel,fiSecondaryDesc,fiModelSeries,fiModelDescriptor,ProductSize,fiProductClassDesc,state,ProductGroup,ProductGroupDesc,Drive_System,Enclosure,Forks,Pad_Type,Ride_Control,Stick,Transmission,Turbocharged,Blade_Extension,Blade_Width,Enclosure_Type,Engine_Horsepower,Hydraulics,Pushblock,Ripper,Scarifier,Tip_Control,Tire_Size,Coupler,Coupler_System,Grouser_Tracks,Hydraulics_Flow,Track_Type,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
393123,6260878,13500,1799594,4102,149,2.0,1000,NaN,NaN,2011-12-30,D4C,D4,C,NaN,NaN,NaN,"Track Type Tractor, Dozer - 75.0 to 85.0 Horse...",New York,TTT,Track Type Tractors,NaN,OROPS,NaN,NaN,NaN,NaN,Standard,NaN,NaN,NaN,NaN,NaN,2 Valve,NaN,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None or Unspecified,PAT,None or Unspecified,NaN,NaN
397987,6288376,9750,1872596,4875,149,2.0,1000,NaN,NaN,2011-12-30,520C,520,C,NaN,NaN,Medium,Wheel Loader - 135.0 to 150.0 Horsepower,New York,WL,Wheel Loader,NaN,EROPS,None or Unspecified,NaN,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 Valve,NaN,NaN,NaN,NaN,None or Unspecified,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
392664,6258093,14500,1877553,3170,149,2.0,1988,NaN,NaN,2011-12-30,580K,580,K,NaN,NaN,NaN,Backhoe Loader - 14.0 to 15.0 Ft Standard Digg...,New York,BL,Backhoe Loaders,Two Wheel Drive,EROPS,None or Unspecified,None or Unspecified,No,Extended,Standard,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
400458,6315563,12500,1869637,26456,149,2.0,2010,NaN,NaN,2011-12-30,L160,L160,NaN,NaN,NaN,Compact,Wheel Loader - 40.0 to 60.0 Horsepower,New York,WL,Wheel Loader,NaN,OROPS,None or Unspecified,NaN,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 Valve,NaN,NaN,NaN,NaN,None or Unspecified,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
400217,6312170,11000,1812622,9580,149,2.0,2005,NaN,NaN,2011-12-30,T190,T190,NaN,NaN,NaN,NaN,Skid Steer Loader - 1751.0 to 2201.0 Lb Operat...,New York,SSL,Skid Steer Loaders,NaN,OROPS,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Auxiliary,NaN,NaN,NaN,NaN,NaN,None or Unspecified,None or Unspecified,None or Unspecified,Standard,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# To extract information from 'saledate'
add_datepart(df_raw, 'saledate')
# Maps string categorical variables to integers
train_cats(df_raw)
# Ordering values associated with 'ProductSize'
df_raw['ProductSize'] = (df_raw['ProductSize'].cat.
                        reorder_categories(['Compact', 'Mini', 'Small', 'Medium',
                                            'Large / Medium', 'Large']))
# Deals with missing values and numericalizes the string columns 
df, y, nas = proc_df(df_raw, y_fld='SalePrice')
# Kaggle's evaluation metric is RMSLE (root mean squared log error) so we can just predict the log of the prices and take the RMSE
y = np.log(y)

In [ ]:
df.sample(5)

### Choosing a Validation Set

In [ ]:
# Splits the data into two sets, the first set has n_trn rows
def split_valid(a, n_trn):
  return a[:n_trn], a[n_trn:]

In [ ]:
n_tst = 12000
n_trn = len(df) - n_tst
X, X_test = split_valid(df.copy(), n_trn)
y, y_test = split_valid(y, n_trn)

NameError: ignored

In [ ]:
X_test.head()

In [ ]:
X_test.tail()

In [ ]:
# Splits the data into a validation set given a specific time interval
def split_valid_saledate(X, y, yr_start, m_start, d_start, yr_end, m_end, d_end):
  # Find the indices of the beginning and end rows of our validation set
  valid_start = X[(X['saleYear'] == yr_start) & (X['saleMonth'] == m_start) & (X['saleDay'] == d_start)].index[0]
  valid_end = X[(X['saleYear'] == yr_end) & (X['saleMonth'] == m_end) & (X['saleDay'] == d_end)].index[-1]
  # The indices of the validation rows
  valid_ind = np.arange(valid_start, valid_end+1)
  X_train, X_valid = X.loc[:valid_start-1], X.loc[valid_ind]
  y_train, y_valid = y[:valid_start], y[valid_ind]
  return X_train, y_train, X_valid, y_valid

In [ ]:
# The accuracy of different models on a validation set
def valid_accuracy(X_train, y_train, X_valid, y_valid):
  # Different hyperparamters for our model
  hyper_params = [
                  {},
                  {'min_samples_leaf': 3},
                  {'max_features': 0.5},
                  {'max_features': 0.5, 'bootstrap': False},
                  {'min_samples_leaf': 3, 'max_features': 0.5}
  ]
  # The scores of different models
  sc = []
  for curr_hype in hyper_params:
    # Create a model using the current hyperparameters 
    m = RandomForestRegressor(n_jobs=-1, **curr_hype)
    m.fit(X_train, y_train)
    # Add the accuracy of our model on the validation set
    sc.append(m.score(X_valid, y_valid))
  return sc

In [ ]:
# To quicken things up
set_rf_samples(50000)

In [ ]:
# First, we need to reset X's index
X.reset_index(inplace=True)
X.drop('index', axis=1, inplace=True)

In [ ]:
# The different intervals for our validation set
intervals = [
             [2011, 5, 10, 2011, 8, 18],
             [2011, 4, 19, 2011, 7, 29],
             [2010, 8, 19, 2010, 12, 30]
]
# The test set's score
test_score = valid_accuracy(X, y, X_test, y_test)
# Go through every interval
for curr_inter in intervals:
  print(curr_inter)
  # Extract the current validation set and find its score
  curr_X_train, curr_y_train, curr_X_valid, curr_y_valid = split_valid_saledate(X, y, *curr_inter)
  curr_valid_score = valid_accuracy(curr_X_train, curr_y_train, curr_X_valid, curr_y_valid)
  print(sum(abs(np.array(curr_valid_score) - np.array(test_score)))/5)

In [ ]:
X_train, y_train, X_valid, y_valid = split_valid_saledate(X, y, 2010, 8, 19, 2010, 12, 30)

### Basic Hypertuning

In [ ]:
def rmse(x, y): return math.sqrt(((x-y)**2).mean())

In [ ]:
def print_score(m, x_trn, y_trn, x_vld, y_vld):
  r = [rmse(m.predict(x_trn), y_trn), rmse(m.predict(x_vld), y_vld)]
  print(r)

In [ ]:
# We're using only the last 50000 rows of our training data 
# to speed up the process of training. Using set_rf_samples(50000)
# on the entire dataset yielded a slightly worse accuracy.
m = RandomForestRegressor(n_jobs=-1, n_estimators=40)
m.fit(X_train.iloc[-50000:], y_train[-50000:])
print_score(m, X_train.iloc[-50000:], y_train[-50000:], X_valid, y_valid)

In [ ]:
m = RandomForestRegressor(n_jobs=-1, n_estimators=40, min_samples_leaf=3)
m.fit(X_train.iloc[-50000:], y_train[-50000:])
print_score(m, X_train.iloc[-50000:], y_train[-50000:], X_valid, y_valid)

In [ ]:
m = RandomForestRegressor(n_jobs=-1, n_estimators=40, min_samples_leaf=5)
m.fit(X_train.iloc[-50000:], y_train[-50000:])
print_score(m, X_train.iloc[-50000:], y_train[-50000:], X_valid, y_valid)

In [ ]:
m = RandomForestRegressor(n_jobs=-1, n_estimators=40, min_samples_leaf=10)
m.fit(X_train.iloc[-50000:], y_train[-50000:])
print_score(m, X_train.iloc[-50000:], y_train[-50000:], X_valid, y_valid)

In [ ]:
m = RandomForestRegressor(n_jobs=-1, n_estimators=40, min_samples_leaf=5, max_features=0.5)
m.fit(X_train.iloc[-50000:], y_train[-50000:])
print_score(m, X_train.iloc[-50000:], y_train[-50000:], X_valid, y_valid)

In [ ]:
m = RandomForestRegressor(n_jobs=-1, n_estimators=40, min_samples_leaf=5, max_features='sqrt')
m.fit(X_train.iloc[-50000:], y_train[-50000:])
print_score(m, X_train.iloc[-50000:], y_train[-50000:], X_valid, y_valid)

### Feature Selection

In [ ]:
m = RandomForestRegressor(n_jobs=-1, n_estimators=40, min_samples_leaf=5, max_features=0.5)
m.fit(X_train.iloc[-50000:], y_train[-50000:])
print_score(m, X_train.iloc[-50000:], y_train[-50000:], X_valid, y_valid)

[0.15247481569832774, 0.2338587992371092]


In [ ]:
fi = rf_feat_importance(m, X_train)
fi

,cols,imp
13,ProductSize,0.233801
5,YearMade,0.175394
37,Coupler_System,0.112694
10,fiSecondaryDesc,0.055376
8,fiModelDesc,0.048609
...,...,...
59,saleIs_quarter_end,0.000033
58,saleIs_month_start,0.000026
60,saleIs_quarter_start,0.000007
61,saleIs_year_end,0.000000


In [ ]:
def plot_feat_imp(fi):
  fi[:10].set_index('cols').plot.barh()

In [ ]:
plot_feat_imp(fi)

In [ ]:
to_keep = fi[fi.imp >= 0.001].cols
X_keep = X[to_keep].copy()
X_train_keep, X_valid_keep = X_train[to_keep].copy(), X_valid[to_keep].copy()

In [ ]:
m = RandomForestRegressor(n_jobs=-1, n_estimators=40, min_samples_leaf=5, max_features=0.6)
m.fit(X_tarain_keep.iloc[-50000:], y_train[-50000:])
print_score(m, X_train_keep.iloc[-50000:], y_train[-50000:], X_valid_keep, y_valid)

In [ ]:
from scipy.cluster import hierarchy as hc

In [ ]:
corr = np.round(scipy.stats.spearmanr(X_keep).correlation, 4)
corr_condensed = hc.distance.squareform(1-corr)
z = hc.linkage(corr_condensed, method='average')
fig = plt.figure(figsize=(16,10))
dendrogram = hc.dendrogram(z, labels=X_keep.columns, orientation='left', leaf_font_size=16)
plt.show()

In [ ]:
correlated = ['Hydraulics_Flow', 'Grouser_Tracks', 'Coupler_System', 
              'ProductGroupDesc', 'ProductGroup', 'fiBaseModel', 'fiModelDesc',
              'Pushblock', 'Blade_Width', 'Scarifier' 
             ]
for i in correlated:
  m = RandomForestRegressor(n_jobs=-1, n_estimators=40, min_samples_leaf=5, max_features=0.6)
  m.fit(X_train_keep.iloc[-50000:].drop(i, axis=1), y_train[-50000:])
  print(i)
  print_score(m, X_train_keep.iloc[-50000:].drop(i, axis=1), y_train[-50000:], X_valid_keep.drop(i, axis=1), y_valid)

In [ ]:
to_drop = ['Grouser_Tracks', 'ProductGroupDesc', 'Scarifier', 'Pushblock']

In [ ]:
m = RandomForestRegressor(n_jobs=-1, n_estimators=40, min_samples_leaf=5, max_features=0.6)
m.fit(X_train_keep.iloc[-50000:].drop(to_drop, axis=1), y_train[-50000:])
print_score(m, X_train_keep.iloc[-50000:].drop(to_drop, axis=1), y_train[-50000:], X_valid_keep.drop(to_drop, axis=1), y_valid)

In [ ]:
X_keep.drop(to_drop, axis=1, inplace=True)
X_train_keep.drop(to_drop, axis=1, inplace=True)
X_valid_keep.drop(to_drop, axis=1, inplace=True)

### Thorough Analysis of Important Features

In [ ]:
m = RandomForestRegressor(n_jobs=-1, n_estimators=40, min_samples_leaf=5, max_features=0.6)
m.fit(X_train_keep.iloc[-50000:], y_train[-50000:])
print_score(m, X_train_keep.iloc[-50000:], y_train[-50000:], X_valid_keep, y_valid)

In [ ]:
fi = rf_feat_importance(m, X_train_keep)
fi

In [ ]:
X_keep[['YearMade']].describe()

In [ ]:
# Replace all instances of 'YearMade' where its value is equal to 1000
# with 1950
X_keep.loc[X_keep['YearMade'] == 1000, 'YearMade'] = 1950
X_train_keep.loc[X_train_keep['YearMade'] == 1000, 'YearMade'] = 1950
X_valid_keep.loc[X_valid_keep['YearMade'] == 1000, 'YearMade'] = 1950

In [ ]:
m = RandomForestRegressor(n_jobs=-1, n_estimators=40, min_samples_leaf=5, max_features=0.6)
m.fit(X_train_keep.iloc[-50000:], y_train[-50000:])
print_score(m, X_train_keep.iloc[-50000:], y_train[-50000:], X_valid_keep, y_valid)

In [ ]:
X_keep['age'] = X['saleYear']-X['YearMade']
X_train_keep['age'] = X_train['saleYear']-X_train['YearMade']
X_valid_keep['age'] = X_valid['saleYear']-X_valid['YearMade']

In [ ]:
m = RandomForestRegressor(n_jobs=-1, n_estimators=40, min_samples_leaf=5, max_features=0.6)
m.fit(X_train_keep.iloc[-50000:], y_train[-50000:])
print_score(m, X_train_keep.iloc[-50000:], y_train[-50000:], X_valid_keep, y_valid)

In [ ]:
X_keep.drop('age', axis=1, inplace=True)
X_train_keep.drop('age', axis=1, inplace=True)
X_valid_keep.drop('age', axis=1, inplace=True)

In [ ]:
X_keep['Coupler_System'].value_counts()

In [ ]:
X_keep['Coupler_System'].value_counts().plot(kind='bar')

In [ ]:
one_hot_coupler_sys_trn = pd.get_dummies(X_train_keep['Coupler_System'], prefix='Coup_Sys_is')
one_hot_coupler_sys_vld = pd.get_dummies(X_valid_keep['Coupler_System'], prefix='Coup_Sys_is')
one_hot_coupler_sys_trn.head()

In [ ]:
# x_trn and x_vld are copies of X_train_keep and
# X_valid_keep respectively so if something we 
# did caused our accuracy to decrease, we'd 
# still have our original dataset
x_trn, x_vld = X_train_keep.copy(), X_valid_keep.copy()
x_trn.drop('Coupler_System', axis=1, inplace=True)
x_vld.drop('Coupler_System', axis=1, inplace=True)
x_trn = x_trn.join(one_hot_coupler_sys_trn)
x_vld = x_vld.join(one_hot_coupler_sys_vld)

In [ ]:
m = RandomForestRegressor(n_jobs=-1, n_estimators=40, min_samples_leaf=5, max_features=0.6)
m.fit(x_trn.iloc[-50000:], y_train[-50000:])
print_score(m, x_trn.iloc[-50000:], y_train[-50000:], x_vld, y_valid)

In [ ]:
X_keep['ProductSize'].value_counts()

In [ ]:
X_keep['ProductSize'].value_counts().plot(kind='bar')

In [ ]:
X_keep['Enclosure'].value_counts()

In [ ]:
X_keep['Enclosure'].value_counts().plot(kind='bar')

In [ ]:
df_raw['Enclosure'].cat.categories

In [ ]:
X_keep['fiProductClassDesc'].value_counts()

In [ ]:
df_raw['fiProductClassDesc'].value_counts()

### Extrapolation

In [ ]:
# If y_ext[i] is 0, the X_keep[i] is in the training set
y_ext = np.ones(len(X_keep))
y_ext[:len(X_train_keep)] = 0

In [ ]:
m = RandomForestClassifier(n_jobs=-1, n_estimators=40, min_samples_leaf=5, max_features=0.6,
                           oob_score=True)
m.fit(X_keep, y_ext)
m.oob_score_

In [ ]:
fi = rf_feat_importance(m, X_keep)
fi.head(3)

In [ ]:
to_drop = ['saleElapsed', 'SalesID', 'MachineID']

In [ ]:
# See how removing each feature affects our model
for f in to_drop:
  m = RandomForestRegressor(n_jobs=-1, n_estimators=40, min_samples_leaf=5, max_features=0.6)
  m.fit(X_train_keep.drop(f, axis=1).iloc[-50000:], y_train[-50000:])
  print_score(m, X_train_keep.drop(f, axis=1).iloc[-50000:], y_train[-50000:], 
              X_valid_keep.drop(f, axis=1), y_valid)

In [ ]:
to_drop = ['SalesID', 'MachineID']

In [ ]:
X_keep.drop(to_drop, axis=1, inplace=True)
X_train_keep.drop(to_drop, axis=1, inplace=True)
X_valid_keep.drop(to_drop, axis=1, inplace=True)

In [ ]:
m = RandomForestRegressor(n_jobs=-1, n_estimators=40, min_samples_leaf=5, max_features=0.6)
m.fit(X_train_keep.iloc[-50000:], y_train[-50000:])
print_score(m, X_train_keep.iloc[-50000:], y_train[-50000:], 
            X_valid_keep, y_valid)

### Final Model

In [ ]:
reset_rf_samples()

In [ ]:
m = RandomForestRegressor(n_jobs=-1, n_estimators=100, min_samples_leaf=3, max_features=0.6)
m.fit(X_train_keep.iloc[-100000:], y_train[-100000:])
print_score(m, X_train_keep.iloc[-100000:], y_train[-100000:], X_valid_keep., y_valid)

In [ ]:
X_test = X_test[X_keep.columns]

In [ ]:
m = RandomForestRegressor(n_jobs=-1, n_estimators=160, min_samples_leaf=3, max_features=0.6)
m.fit(X_keep, y)
print_score(m, X_keep, y, X_test, y_test)

### XGBoost (optional)

In [ ]:
!pip install xgboost

In [ ]:
from xgboost import XGBRegressor

In [ ]:
m = XGBRegressor(n_jobs=-1, n_estimators=400)
m.fit(X_train_keep.iloc[-50000:], y_train[-50000:])
print_score(m, X_train_keep.iloc[-50000:], y_train[-50000:], 
            X_valid_keep, y_valid)

[12:56:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0.24387817164103584, 0.24787442898203998]


In [ ]:
m = XGBRegressor(n_jobs=-1, n_estimators=400, max_depth=4)
m.fit(X_train_keep.iloc[-50000:], y_train[-50000:])
print_score(m, X_train_keep.iloc[-50000:], y_train[-50000:], 
            X_valid_keep, y_valid)

[23:11:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0.2187821556894904, 0.23327781729938218]


In [ ]:
m = XGBRegressor(n_jobs=-1, n_estimators=400, max_depth=5)
m.fit(X_train_keep.iloc[-50000:], y_train[-50000:])
print_score(m, X_train_keep.iloc[-50000:], y_train[-50000:], 
            X_valid_keep, y_valid)

[23:13:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0.198054141118267, 0.22814543918471594]


In [ ]:
m = XGBRegressor(n_jobs=-1, n_estimators=400, max_depth=6)
m.fit(X_train_keep.iloc[-50000:], y_train[-50000:])
print_score(m, X_train_keep.iloc[-50000:], y_train[-50000:], 
            X_valid_keep, y_valid)

[23:14:40] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0.17614614409158405, 0.22391425919166275]


In [ ]:
m = XGBRegressor(n_jobs=-1, n_estimators=400, max_depth=7)
m.fit(X_train_keep.iloc[-50000:], y_train[-50000:])
print_score(m, X_train_keep.iloc[-50000:], y_train[-50000:], 
            X_valid_keep, y_valid)

[23:15:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0.15277599041142365, 0.22488769807601508]


In [ ]:
m = XGBRegressor(n_jobs=-1, n_estimators=400, max_depth=6, learning_rate=0.01)
m.fit(X_train_keep.iloc[-50000:], y_train[-50000:])
print_score(m, X_train_keep.iloc[-50000:], y_train[-50000:], 
            X_valid_keep, y_valid)

[13:05:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0.31153897565259187, 0.3277399340701414]


In [ ]:
m = XGBRegressor(n_jobs=-1, n_estimators=400, max_depth=6, learning_rate=0.03)
m.fit(X_train_keep.iloc[-50000:], y_train[-50000:])
print_score(m, X_train_keep.iloc[-50000:], y_train[-50000:], 
            X_valid_keep, y_valid)

[23:19:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0.2194946513480657, 0.2380608398396556]


In [ ]:
# We've already done learning_rate=0.1 
m = XGBRegressor(n_jobs=-1, n_estimators=400, max_depth=6, learning_rate=0.3)
m.fit(X_train_keep.iloc[-50000:], y_train[-50000:])
print_score(m, X_train_keep.iloc[-50000:], y_train[-50000:], 
            X_valid_keep, y_valid)

[13:07:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0.13076675980422292, 0.24103057058707475]


In [ ]:
m = XGBRegressor(n_jobs=-1, n_estimators=400, max_depth=6, colsample_bytree=0.5)
m.fit(X_train_keep.iloc[-50000:], y_train[-50000:])
print_score(m, X_train_keep.iloc[-50000:], y_train[-50000:], 
            X_valid_keep, y_valid)

[23:20:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0.17825130301917053, 0.22234699135557273]


In [ ]:
m = XGBRegressor(n_jobs=-1, n_estimators=400, max_depth=6, colsample_bytree=int(X_keep.shape[1]**0.5)/X_keep.shape[1])
m.fit(X_train_keep.iloc[-50000:], y_train[-50000:])
print_score(m, X_train_keep.iloc[-50000:], y_train[-50000:], 
            X_valid_keep, y_valid)

[23:21:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0.19238450368917345, 0.22521114084710347]


### Extra Trees (optional)

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor

In [ ]:
m = ExtraTreesRegressor(n_jobs=-1, n_estimators=40)
m.fit(X_train_keep.iloc[-50000:], y_train[-50000:])
print_score(m, X_train_keep.iloc[-50000:], y_train[-50000:], 
            X_valid_keep, y_valid)

[0.02283931776591394, 0.23636459052640532]


In [ ]:
m = ExtraTreesRegressor(n_jobs=-1, n_estimators=40, min_samples_leaf=3)
m.fit(X_train_keep.iloc[-50000:], y_train[-50000:])
print_score(m, X_train_keep.iloc[-50000:], y_train[-50000:], 
            X_valid_keep, y_valid)

[0.10365802314498147, 0.23309803681034955]


In [ ]:
m = ExtraTreesRegressor(n_jobs=-1, n_estimators=40, min_samples_leaf=5)
m.fit(X_train_keep.iloc[-50000:], y_train[-50000:])
print_score(m, X_train_keep.iloc[-50000:], y_train[-50000:], 
            X_valid_keep, y_valid)

[0.14800774729137847, 0.23494679983383124]


### Ensemble (optional)

In [ ]:
rf = RandomForestRegressor(n_jobs=-1, n_estimators=40, min_samples_leaf=5, max_features=0.6)
rf.fit(X_train_keep.iloc[-50000:], y_train[-50000:])

In [ ]:
xg = XGBRegressor(n_jobs=-1, n_estimators=400, max_depth=6, colsample_bytree=0.5)
xg.fit(X_train_keep.iloc[-50000:], y_train[-50000:])

In [ ]:
xtra = ExtraTreesRegressor(n_jobs=-1, n_estimators=40, min_samples_leaf=3)
xtra.fit(X_train_keep.iloc[-50000:], y_train[-50000:])

In [ ]:
class EnsModel:
  def __init__(self, models, weights=None):
    self.n = len(models)
    self.models = models
    self.weights = weights
    # If weights is empty, we'll simply use the arithmetic mean
    if self.weights is None: self.weights = [(1/self.n)]*self.n

  def predict(self, x):
    pred = self.weights[0]*self.models[0].predict(x)
    # Calculate the weighted average
    for i in range(1, self.n): pred += self.weights[i]*self.models[i].predict(x)
    return pred

In [ ]:
green = EnsModel([xtra, rf], [0.1, 0.9])
print_score(green, X_train_keep.iloc[-50000:], y_train[-50000:], 
            X_valid_keep, y_valid)

[0.14779657489244005, 0.231232724332311]


In [ ]:
green = EnsModel([xtra, rf], [0.3, 0.7])
print_score(green, X_train_keep.iloc[-50000:], y_train[-50000:], 
            X_valid_keep, y_valid)

[0.13640606670371266, 0.22989098025645316]


In [ ]:
green = EnsModel([xtra, rf], [0.5, 0.5])
print_score(green, X_train_keep.iloc[-50000:], y_train[-50000:], 
            X_valid_keep, y_valid)

[0.12573981332874673, 0.2295554451824303]


In [ ]:
green = EnsModel([xtra, rf], [0.7, 0.3])
print_score(green, X_train_keep.iloc[-50000:], y_train[-50000:], 
            X_valid_keep, y_valid)

[0.11599777763840023, 0.23023051846146536]


In [ ]:
green = EnsModel([xtra, rf], [0.5, 0.5])
m = EnsModel([green, xg], [0.1, 0.9])
print_score(m, X_train_keep.iloc[-50000:], y_train[-50000:], 
            X_valid_keep, y_valid)

[0.17088884534584686, 0.22010198742097675]


In [ ]:
m = EnsModel([green, xg], [0.3, 0.7])
print_score(m, X_train_keep.iloc[-50000:], y_train[-50000:], 
            X_valid_keep, y_valid)

[0.158211349214013, 0.21918565560178502]


In [ ]:
m = EnsModel([green, xg], [0.5, 0.5])
print_score(m, X_train_keep.iloc[-50000:], y_train[-50000:], 
            X_valid_keep, y_valid)

[0.14677235714641265, 0.22002187084892433]


In [ ]:
rf = RandomForestRegressor(n_jobs=-1, n_estimators=160, min_samples_leaf=5, max_features=0.6)
rf.fit(X_keep, y)

In [ ]:
xg = XGBRegressor(n_jobs=-1, n_estimators=1600, max_depth=6, colsample_bytree=0.5)
xg.fit(X_keep, y)

In [ ]:
xtra = ExtraTreesRegressor(n_jobs=-1, n_estimators=160, min_samples_leaf=3)
xtra.fit(X_keep, y)

In [ ]:
green = EnsModel([xtra, rf], [0.5, 0.5])
m = EnsModel([green, xg], [0.3, 0.7])
print_score(m, X_keep.iloc[-100000:], y[-100000:], X_test, y_test)

### Neural Net

In [ ]:
!pip install fastai>=1.0.0

In [ ]:
from fastai.tabular import *

In [ ]:
df_raw = pd.read_csv('Train.csv', low_memory=False, parse_dates=['saledate'])
df_raw.sort_values('saledate', inplace=True)
df_raw['SalePrice'] = np.log(df_raw['SalePrice'])
add_datepart(df_raw, 'saledate')
df_train = df_raw.iloc[:-12000].copy()

In [ ]:
cols = ['Undercarriage_Pad_Width', 'Stick_Length', 'YearMade', 'saleYear', 'saleMonth',
                'saleWeek', 'saleDay', 'saleDayofweek',
                'saleDayofyear']
df_train_cols, df_raw_cols = df_train[cols].copy(), df_raw[cols].copy()

In [ ]:
df_train_cols.nunique()

Undercarriage_Pad_Width     19
Stick_Length                29
YearMade                    71
saleYear                    23
saleMonth                   12
saleWeek                    53
saleDay                     31
saleDayofweek                7
saleDayofyear              360
dtype: int64

In [ ]:
df_raw_cols.nunique()

Undercarriage_Pad_Width     19
Stick_Length                29
YearMade                    72
saleYear                    23
saleMonth                   12
saleWeek                    53
saleDay                     31
saleDayofweek                7
saleDayofyear              360
dtype: int64

In [ ]:
dep_var = 'SalePrice'
cat_vars = list(set(df_raw.columns)-set(['SalePrice',
                                     'SalesID',
                                     'MachineHoursCurrentMeter',
                                     'saleElapsed']))
cont_vars = ['MachineHoursCurrentMeter', 'saleElapsed']

In [ ]:
# Sample data for hyperparameter tuning
n_sample = 100000
small_train = df_train.iloc[-n_sample:]

In [ ]:
procs = [FillMissing, Categorify, Normalize]
# The indices of our validation rows
n_valid = 12000
valid_idx = list(range(n_sample-n_valid, n_sample)) # This is equal to [88000, 88001, ..., 99999]

In [ ]:
small_data = (TabularList.from_df(small_train, cat_names=cat_vars, cont_names=cont_vars, procs=procs)
        .split_by_idx(valid_idx)
        .label_from_df(dep_var)
        .databunch())

In [ ]:
layers = [500]
metrics = [rmse]
y_min, y_max = 0.8*min(small_train[dep_var]), 1.2*max(small_train[dep_var])
y_range = (y_min, y_max)
ps = [0]
emb_drop = 0
wd = 1e-1

In [ ]:
learn = tabular_learner(small_data, layers=layers, 
                        metrics=metrics, ps=ps,
                        emb_drop=emb_drop, y_range=y_range, wd=wd)

In [ ]:
def plot_lr_loss():
  learn.lr_find(start_lr=1e-4, end_lr=1)
  learn.recorder.plot()

In [ ]:
plot_lr_loss()

In [ ]:
lr = 3e-4
learn.fit_one_cycle(10, lr)

In [ ]:
ps = [1e-1]
learn = tabular_learner(small_data, layers=layers, 
                        metrics=metrics, ps=ps,
                        emb_drop=emb_drop, y_range=y_range, wd=wd)

In [ ]:
plot_lr_loss()

In [ ]:
lr = 2e-4
learn.fit_one_cycle(10, lr)

In [ ]:
ps = [3e-1]
learn = tabular_learner(small_data, layers=layers, 
                        metrics=metrics, ps=ps,
                        emb_drop=emb_drop, y_range=y_range, wd=wd)

In [ ]:
plot_lr_loss()

In [ ]:
lr = 3e-4
learn.fit_one_cycle(10, lr)

In [ ]:
ps = [1e-1]

In [ ]:
emb_drop = 1e-2
learn = tabular_learner(small_data, layers=layers, 
                        metrics=metrics, ps=ps,
                        emb_drop=emb_drop, y_range=y_range, wd=wd)

In [ ]:
plot_lr_loss()

In [ ]:
lr = 5e-4
learn.fit_one_cycle(10, lr)

In [ ]:
emb_drop = 3e-2
learn = tabular_learner(small_data, layers=layers, 
                        metrics=metrics, ps=ps,
                        emb_drop=emb_drop, y_range=y_range, wd=wd)

In [ ]:
plot_lr_loss()

In [ ]:
lr = 3e-4
learn.fit_one_cycle(10, lr)

In [ ]:
emb_drop = 1e-2

In [ ]:
n_sample = 150000
n_test = 12000
total = n_sample+n_test
# Contains our training (first 150000 rows) and test (last 12000 rows) sets. 
train_test = df_raw.iloc[-total:]
test_idx = list(range(total-n_test, total)) # This is equal to [150000, 150001, ..., 161999]
data = (TabularList.from_df(train_test, cat_names=cat_vars, cont_names=cont_vars, procs=procs)
        .split_by_idx(test_idx)
        .label_from_df(dep_var)
        .databunch())

In [ ]:
learn = tabular_learner(data, layers=layers, 
                        metrics=metrics, ps=ps,
                        emb_drop=emb_drop, y_range=y_range, wd=wd)

In [ ]:
plot_lr_loss()

In [ ]:
lr = 4e-4
learn.fit_one_cycle(10, lr)